In [ ]:
"""
    This is not a necessary step. However, if you want to crop the scale bar or legend out of a set of images. You can use these.


"""

In [ ]:
"""
    This project provides a set of Python scripts to crop a specific region from a set of images using OpenCV. The key functionality includes:

    Image Selection and Cropping:
        select_crop_region(image_path): Allows the user to interactively select a region of interest (ROI) from a reference image.
        crop_image(image_path, crop_region): Crops the selected region from each image in the list.

    Main Function:
        The script iterates over multiple images, applies the same crop region, and saves the cropped images, replacing the originals.

Usage:

    Define the paths to your images.
    Run the script.
    Select the region of interest from the first image.
    All images will be cropped accordingly.

"""

In [1]:
%matplotlib

Using matplotlib backend: <object object at 0x11198f510>


# Crop

In [1]:
import cv2
import os

def select_crop_region(image_path):
    # Load the image
    image = cv2.imread(image_path)
    
    # Check if the image is loaded correctly
    if image is None:
        print(f"Error: Unable to load image at {image_path}")
        return None
    
    # Create a named window with adjustable size
    window_name = "Select Crop Region"
    cv2.namedWindow(window_name, cv2.WINDOW_NORMAL)
    cv2.resizeWindow(window_name, 800, 600)
    
    # Select ROI (Region of Interest)
    print("Select the crop region and press Enter or Space to confirm.")
    r = cv2.selectROI(window_name, image, fromCenter=False, showCrosshair=True)
    
    # Check if a valid region is selected
    if r[2] == 0 or r[3] == 0:
        print("Error: No valid region selected.")
        return None
    
    # Close the window and clean up
    cv2.destroyWindow(window_name)
    print(f"Crop region selected: {r}")
    
    # Return the selected region (x, y, w, h)
    return r

def crop_image(image_path, crop_region):
    # Load the image
    image = cv2.imread(image_path)
    
    # Check if the image is loaded correctly
    if image is None:
        print(f"Error: Unable to load image at {image_path}")
        return None
    
    # Extract the crop region
    x, y, w, h = crop_region
    cropped_image = image[y:y+h, x:x+w]
    
    # Save the cropped image, replacing the original
    cv2.imwrite(image_path, cropped_image)
    print(f"Cropped image saved: {image_path}")
    
    return image_path

def list_images(folder_path):
    # List all images in the folder
    image_files = [f for f in os.listdir(folder_path) if f.lower().endswith(('.png', '.jpg', '.jpeg', '.bmp', '.tiff'))]
    print("Images in the folder:")
    for image in image_files:
        print(image)
    return image_files

def batch_crop_images(folder_path, crop_region):
    # List and print out all images in the folder
    image_files = list_images(folder_path)
    
    # Apply the crop to each image in the folder
    for filename in image_files:
        file_path = os.path.join(folder_path, filename)
        crop_image(file_path, crop_region)
        print(f"Cropped {filename} using the selected region.")

In [4]:
# Usage example:
# List all images in the folder first
folder_path = "To_be_Aligned"
list_images(folder_path)

Images in the folder:
USU-4169B 250-355 Al Map.jpeg
USU-4169B 250-355 Ca Map.jpeg
USU-4169B 250-355 K Map.jpeg
USU-4169B 250-355 Map.bmp
USU-4169B 250-355 Na Map.jpeg
USU-4169B 250-355 Si Map.jpeg


['USU-4169B 250-355 Al Map.jpeg',
 'USU-4169B 250-355 Ca Map.jpeg',
 'USU-4169B 250-355 K Map.jpeg',
 'USU-4169B 250-355 Map.bmp',
 'USU-4169B 250-355 Na Map.jpeg',
 'USU-4169B 250-355 Si Map.jpeg']

If your single elementary images have scale bars need to be cropped out together, you can put the composite image in the `SafeRoom` folder and run the following script.

In [3]:
# Select crop region for a sample image
sample_image_path = r"To_be_Aligned\USU-4169B 250-355 Si Map.jpeg"
crop_region = select_crop_region(sample_image_path)

# If a crop region is selected, apply it to all images in the folder
if crop_region:
    batch_crop_images(folder_path, crop_region)

Select the crop region and press Enter or Space to confirm.
Crop region selected: (317, 0, 1421, 1404)
Images in the folder:
USU-4169B 250-355 Al Map.jpeg
USU-4169B 250-355 Ca Map.jpeg
USU-4169B 250-355 K Map.jpeg
USU-4169B 250-355 Na Map.jpeg
USU-4169B 250-355 Si Map.jpeg
Cropped image saved: To_be_Aligned\USU-4169B 250-355 Al Map.jpeg
Cropped USU-4169B 250-355 Al Map.jpeg using the selected region.
Cropped image saved: To_be_Aligned\USU-4169B 250-355 Ca Map.jpeg
Cropped USU-4169B 250-355 Ca Map.jpeg using the selected region.
Cropped image saved: To_be_Aligned\USU-4169B 250-355 K Map.jpeg
Cropped USU-4169B 250-355 K Map.jpeg using the selected region.
Cropped image saved: To_be_Aligned\USU-4169B 250-355 Na Map.jpeg
Cropped USU-4169B 250-355 Na Map.jpeg using the selected region.
Cropped image saved: To_be_Aligned\USU-4169B 250-355 Si Map.jpeg
Cropped USU-4169B 250-355 Si Map.jpeg using the selected region.


# If the image size in this pair is not equal, or different a lot, run the resize here and then go to the next script to align

In [5]:
import cv2
import os
import numpy as np

# Function to resize and pad/crop an image to match the base image's size
def resize_to_match_base(image_to_align, base_image):
    base_h, base_w = base_image.shape[:2]
    align_h, align_w = image_to_align.shape[:2]

    # If resizing is not needed (same dimensions), return as is
    if base_h == align_h and base_w == align_w:
        return image_to_align, None

    # Calculate aspect ratios
    aspect_ratio_base = base_w / base_h
    aspect_ratio_align = align_w / align_h

    # Resize with aspect ratio preservation
    if aspect_ratio_align > aspect_ratio_base:
        new_w = base_w
        new_h = int((align_h * base_w) / align_w)
    else:
        new_h = base_h
        new_w = int((align_w * base_h) / align_h)

    # Resize the image
    resized_image = cv2.resize(image_to_align, (new_w, new_h), interpolation=cv2.INTER_LINEAR)

    # Pad or crop to fit exact base dimensions
    padded_image = np.zeros((base_h, base_w, 3), dtype=np.uint8)  # 3 channels assumed (BGR)
    y_offset = (base_h - new_h) // 2
    x_offset = (base_w - new_w) // 2
    padded_image[y_offset:y_offset+new_h, x_offset:x_offset+new_w] = resized_image

    return padded_image, (new_w, new_h, y_offset, x_offset)

# Function to apply recorded padding and resizing to the rest of the images
def apply_to_other_images(base_image_path, target_image_path, output_folder):
    # Load the base and target images
    base_image = cv2.imread(base_image_path)
    target_image = cv2.imread(target_image_path)

    if base_image is None or target_image is None:
        print("Error loading one of the images.")
        return

    # Resize and pad/crop the target image to match the base image
    resized_image, resize_params = resize_to_match_base(target_image, base_image)
    
    # Save the resized target image to output
    if not os.path.exists(output_folder):
        os.makedirs(output_folder)
    target_output_path = os.path.join(output_folder, os.path.basename(target_image_path))
    cv2.imwrite(target_output_path, resized_image)
    print(f"Saved resized target image: {target_output_path}")

    # Process and save other images in the folder
    image_folder = os.path.dirname(target_image_path)
    for filename in os.listdir(image_folder):
        file_path = os.path.join(image_folder, filename)
        if file_path == target_image_path or not filename.lower().endswith(('.png', '.jpg', '.jpeg')):
            continue  # Skip the target image itself or non-image files

        image = cv2.imread(file_path)
        if image is None:
            print(f"Error loading image: {file_path}")
            continue

        # Only resize without recording parameters on subsequent images
        resized_image, _ = resize_to_match_base(image, base_image)
        output_image_path = os.path.join(output_folder, filename)
        cv2.imwrite(output_image_path, resized_image)
        print(f"Saved resized image: {output_image_path}")

    print(f"All images processed and saved in directory: {output_folder}")

In [8]:
output_folder = 'To_be_Aligned/SafeRoom'

In [11]:
# Usage example:
base_image_path = r"To_be_Aligned\USU-4162A 250-355 Elemental Map.jpeg"
target_image_path = r"To_be_Aligned\USU-4162A 250-355 Si Map.jpeg"
apply_to_other_images(base_image_path, target_image_path, output_folder)

Saved resized target image: To_be_Aligned/SafeRoom\USU-4162A 250-355 Si Map.jpeg
Saved resized image: To_be_Aligned/SafeRoom\USU-4162A 250-355 Al Map.jpeg
Saved resized image: To_be_Aligned/SafeRoom\USU-4162A 250-355 Ca Map.jpeg
Saved resized image: To_be_Aligned/SafeRoom\USU-4162A 250-355 Elemental Map.jpeg
Saved resized image: To_be_Aligned/SafeRoom\USU-4162A 250-355 Fe Map.jpeg
Saved resized image: To_be_Aligned/SafeRoom\USU-4162A 250-355 K Map.jpeg
Saved resized image: To_be_Aligned/SafeRoom\USU-4162A 250-355 Na Map.jpeg
All images processed and saved in directory: To_be_Aligned/SafeRoom
